In [1]:
import v3io_frames as v3f

In [2]:
# Create a V3IO Frames client for the "users" data container
client = v3f.Client("framesd:8081", container="users")

In [7]:
stream = "admin/examples/stream1"

In [5]:
client.create("stream", table=stream, retention_hours=2)

In [6]:
import numpy as np
df = pd.DataFrame(np.random.rand(9, 3) * 100,
                  columns=["cpu", "mem", "disk"])
client.write("stream", table="admin/examples/stream1", dfs=df)

In [8]:
import requests
import json
import base64
import os
import time

In [12]:
shardId = 0
def seek_location(shardId: int = 0,
                  container: str = '',
                  stream: str = "",
                  url = '',
                 headers = ''):
    payload = {
            "ShardId": shardId,
            "Type": "EARLIEST"
          }
    headers["X-v3io-function"] = "Seek"
    r = requests.post(url, headers=headers,json=payload)
    return json.loads(r.text)['Location']

In [14]:
# Fire up the Kafka Consumer
url = "http://v3io-webapi:8081/%s/%s/"% ('users',stream)
headers = { "Content-Type": "application/json",
        "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
}

location = seek_location(shardId=shardId,
                         container='users',
                         stream=stream,
                         url=url,
                        headers=headers)
img_payload = {
        "ShardId":    shardId,
        "Location":   location,
    "Limit":10
}
headers["X-v3io-function"] = "GetRecords"
r = requests.post(url, headers=headers,json=img_payload)
print(r.text)
response=json.loads(r.text)
for record in response['Records']:
    print(record)


{ "NextLocation":"AQAAAAEAAAAAAID3AAAAAA==","MSecBehindLatest":0,"RecordsBehindLatest":0,"RecordsNum":9,"Records": [{ "SequenceNumber":1,"Data":"eyJjcHUiOjguMTM4NzAwNTkzNjU5NDIsImRpc2siOjIyLjI4MzQyMDg0MTQ4OTYwOCwibWVtIjo4NC4zNDk0NDU1OTcxMTcwNH0=","ArrivalTimeSec":1594153589,"ArrivalTimeNSec":548772920 },{ "SequenceNumber":2,"Data":"eyJjcHUiOjg3LjcyMDc2OTkzMDcwODgsImRpc2siOjY0LjU5NDMyNjA4MTY4Mzc1LCJtZW0iOjUzLjI2NDE1NDk1ODQ0MjN9","ArrivalTimeSec":1594153589,"ArrivalTimeNSec":548772920 },{ "SequenceNumber":3,"Data":"eyJjcHUiOjY1LjE1Nzg3MDQ1Nzc5OTYyLCJkaXNrIjo5NS42ODUyOTkzMTMyNjE1MiwibWVtIjo3MS4zNjkxNjc1MTE2MzM5OH0=","ArrivalTimeSec":1594153589,"ArrivalTimeNSec":548772920 },{ "SequenceNumber":4,"Data":"eyJjcHUiOjk0LjI4ODMxMzgzMDQyMjA0LCJkaXNrIjo0MS45ODc0ODA2ODExNjQ0NSwibWVtIjoxOS40MTUzNzM1MDI2MDAzNDJ9","ArrivalTimeSec":1594153589,"ArrivalTimeNSec":548772920 },{ "SequenceNumber":5,"Data":"eyJjcHUiOjI3LjcyMjEzNjQ4MjM4MTAxLCJkaXNrIjo5Mi45NzQwMTg0MDE3OTk0NywibWVtIjo4Ni40OTQ1ODk1Mjk4ODg3OH0=","